In [0]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import *
from pyspark.sql.functions import *



spark = SparkSession.builder.appName("Interview Questions").getOrCreate()

In [0]:
%sql
CREATE TABLE swiggy_orders (
    orderid INT,
    custid INT,
    city STRING,
    del_partner STRING,
    order_time TIMESTAMP,
    deliver_time TIMESTAMP,
    predicted_time INT
)
USING DELTA;


In [0]:
%sql
INSERT INTO swiggy_orders (orderid, custid, city, del_partner, order_time, deliver_time, predicted_time)
VALUES
(1, 101, 'Mumbai', 'Partner A', '2024-12-18 10:00:00', '2024-12-18 11:30:00', 60),
(2, 102, 'Delhi', 'Partner A', '2024-12-18 09:00:00', '2024-12-18 10:00:00', 45),
(3, 103, 'Pune', 'Partner A', '2024-12-18 15:00:00', '2024-12-18 15:30:00', 30),
(4, 104, 'Mumbai', 'Partner A', '2024-12-18 14:00:00', '2024-12-18 14:50:00', 45),
(5, 105, 'Bangalore', 'Partner B', '2024-12-18 08:00:00', '2024-12-18 08:29:00', 30),
(6, 106, 'Hyderabad', 'Partner B', '2024-12-18 13:00:00', '2024-12-18 14:00:00', 70),
(7, 107, 'Kolkata', 'Partner B', '2024-12-18 10:00:00', '2024-12-18 10:40:00', 45),
(8, 108, 'Delhi', 'Partner B', '2024-12-18 18:00:00', '2024-12-18 18:30:00', 40),
(9, 109, 'Chennai', 'Partner C', '2024-12-18 07:00:00', '2024-12-18 07:40:00', 30),
(10, 110, 'Mumbai', 'Partner C', '2024-12-18 12:00:00', '2024-12-18 13:00:00', 50),
(11, 111, 'Delhi', 'Partner C', '2024-12-18 09:00:00', '2024-12-18 09:35:00', 30),
(12, 112, 'Hyderabad', 'Partner C', '2024-12-18 16:00:00', '2024-12-18 16:45:00', 30);


num_affected_rows,num_inserted_rows
12,12


In [0]:
%sql
SHOW TABLES;


database,tableName,isTemporary
default,swiggy_orders,false


In [0]:
%sql
CREATE TABLE sale_data (
    order_date DATE,
    customer_id INT,
    store_id INT,
    product_id INT,
    sale INT,
    order_value INT
)
USING DELTA;


In [0]:
%sql
INSERT INTO sale_data (order_date, customer_id, store_id, product_id, sale, order_value)
VALUES
('2024-12-01', 109, 1, 3, 2, 700),
('2024-12-02', 110, 2, 2, 1, 300),
('2024-12-03', 111, 1, 5, 3, 900),
('2024-12-04', 112, 3, 1, 2, 500),
('2024-12-05', 113, 3, 4, 4, 1200),
('2024-12-05', 114, 3, 4, 2, 400),
('2024-12-05', 115, 3, 4, 1, 300),
('2024-12-01', 101, 1, 4, 2, 500),
('2024-12-01', 102, 1, 4, 1, 300),
('2024-12-02', 103, 2, 4, 3, 900),
('2024-12-02', 104, 2, 4, 1, 400),
('2024-12-03', 105, 1, 4, 2, 600),
('2024-12-03', 106, 1, 4, 3, 800),
('2024-12-04', 107, 3, 4, 1, 200),
('2024-12-04', 108, 3, 4, 2, 500);


num_affected_rows,num_inserted_rows
15,15


In [0]:
%sql
select * from swiggy_orders

orderid,custid,city,del_partner,order_time,deliver_time,predicted_time
1,101,Mumbai,Partner A,2024-12-18T10:00:00.000+0000,2024-12-18T11:30:00.000+0000,60
2,102,Delhi,Partner A,2024-12-18T09:00:00.000+0000,2024-12-18T10:00:00.000+0000,45
3,103,Pune,Partner A,2024-12-18T15:00:00.000+0000,2024-12-18T15:30:00.000+0000,30
4,104,Mumbai,Partner A,2024-12-18T14:00:00.000+0000,2024-12-18T14:50:00.000+0000,45
5,105,Bangalore,Partner B,2024-12-18T08:00:00.000+0000,2024-12-18T08:29:00.000+0000,30
6,106,Hyderabad,Partner B,2024-12-18T13:00:00.000+0000,2024-12-18T14:00:00.000+0000,70
7,107,Kolkata,Partner B,2024-12-18T10:00:00.000+0000,2024-12-18T10:40:00.000+0000,45
8,108,Delhi,Partner B,2024-12-18T18:00:00.000+0000,2024-12-18T18:30:00.000+0000,40
9,109,Chennai,Partner C,2024-12-18T07:00:00.000+0000,2024-12-18T07:40:00.000+0000,30
10,110,Mumbai,Partner C,2024-12-18T12:00:00.000+0000,2024-12-18T13:00:00.000+0000,50


In [0]:
%sql
select del_partner, count(orderid) AS delayed_orders
from swiggy_orders
where TIMESTAMPDIFF(MINUTE, order_time, deliver_time) > predicted_time
group by del_partner;


del_partner,delayed_orders
Partner C,4
Partner A,3


In [0]:
%sql
SELECT del_partner, 
       COUNT(CASE WHEN TIMESTAMPDIFF(MINUTE, order_time, deliver_time) > predicted_time THEN 1 END) AS delayed_orders
FROM swiggy_orders
GROUP BY del_partner;



del_partner,delayed_orders
Partner B,0
Partner C,4
Partner A,3


In [0]:
%sql
select * from sale_data 

order_date,customer_id,store_id,product_id,sale,order_value
2024-12-01,109,1,3,2,700
2024-12-02,110,2,2,1,300
2024-12-03,111,1,5,3,900
2024-12-04,112,3,1,2,500
2024-12-05,113,3,4,4,1200
2024-12-05,114,3,4,2,400
2024-12-05,115,3,4,1,300
2024-12-01,101,1,4,2,500
2024-12-01,102,1,4,1,300
2024-12-02,103,2,4,3,900


On which date did the 3rd highest sale of product 4 take place in terms of value ?

In [0]:
%sql
WITH RankedSales AS (
    SELECT order_date, order_value,
           ROW_NUMBER() OVER (PARTITION BY product_id ORDER BY order_value DESC) AS sale_rank
    FROM sale_data
    WHERE product_id = 4
)
SELECT order_date
FROM RankedSales
WHERE sale_rank = 3;


order_date
2024-12-03
